In [46]:
import torch

In [1]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
X_train.shape

y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])
y_test = torch.tensor([0, 1])

NameError: name 'torch' is not defined

In [48]:
# Next, we create a custom dataset class, ToyDataset, by subclassing from PyTorch's Dataset parent class
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y
        
    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
    
    def __len__(self):
        return self.labels.shape[0]
    
train_dataset = ToyDataset(X_train, y_train)
test_dataset = ToyDataset(X_test, y_test)

In [49]:
print(len(train_dataset))

5


In [50]:
# Instantiating data loaders
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=2,
    shuffle=False,
    num_workers=0
)

In [51]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])


In [52]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()
        
        self.layers = torch.nn.Sequential(
            
            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),
            
            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),
            
            torch.nn.Linear(20, num_outputs)
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [53]:
# Neural network training in PyTorch
import torch.nn.functional as F

torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2, num_outputs=2)

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr=0.5)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        # forward pass
        features, labels = features.to(device), labels.to(device)
        logits = model(features)
        
        loss = F.cross_entropy(logits, labels)
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # logging
        if not batch_idx % 1:
            print (f"Epoch: {epoch+1:03d}/{num_epochs:03d} | "
                   f"Batch {batch_idx:03d}/{len(train_loader):03d} | "
                   f"Cost: {loss:.2f}")
    model.eval()

Epoch: 001/003 | Batch 000/002 | Cost: 0.75
Epoch: 001/003 | Batch 001/002 | Cost: 0.65
Epoch: 002/003 | Batch 000/002 | Cost: 0.44
Epoch: 002/003 | Batch 001/002 | Cost: 0.13
Epoch: 003/003 | Batch 000/002 | Cost: 0.03
Epoch: 003/003 | Batch 001/002 | Cost: 0.00


In [62]:
model.eval()
with torch.no_grad():
    outputs = model(X_train)
outputs

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])

In [63]:
torch.set_printoptions(sci_mode=False)
probas = torch.softmax(outputs, dim=1)
probas

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])

In [56]:
predictions = torch.argmax(outputs, dim=1)
torch.sum(predictions == y_train)

tensor(5)

In [57]:
def compute_accuracy(model, data_loader):

    model.eval()
    correct = 0.0
    total_examples = 0
    
    for idx, (features, labels) in enumerate(data_loader):
        
        with torch.no_grad():
            logits = model(features)
        
        predictions = torch.argmax(logits, dim=1)
        compare = predictions == labels
        correct += torch.sum(compare)
        total_examples += len(compare)
        
    return (correct / total_examples).item()

In [61]:
compute_accuracy(model, test_loader)

1.0

In [59]:
# Saving and loading models
torch.save(model.state_dict(), "model.pth")

In [60]:
model = NeuralNetwork(2, 2)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>